In [1]:
import numpy as np
from model import *
import datetime
from utils import *
from preprocessing import *
import random
import glob
from os.path import join, getctime, basename
from load_data import load_data
from save_model import replace_model_if_better
from keras.models import load_model
from shutil import rmtree

## Loading Preprocessed Data

# Create result folders 
save_path = join(get_save_path(), 'ConvMixGRU')
new_path = join(save_path, 'best_val')
time = str(datetime.datetime.now()).replace(' ', '_')
workpath = join(save_path, time)
ensure_dir(new_path)
ensure_dir(workpath)
ensure_dir(save_path)

# Load preprocessed data
aug_data = load_data_pickle(get_save_path())
total_X_test = aug_data['total_X_test']
total_y_test = aug_data['total_y_test']
total_X_train = aug_data['total_X_train']
total_y_train = aug_data['total_y_train']
total_X_val = aug_data['total_X_val']
total_y_val = aug_data['total_y_val']
total_X_train = np.transpose(total_X_train, (0, 2, 1))
total_X_val = np.transpose(total_X_val, (0, 2, 1))
total_X_test = np.transpose(total_X_test, (0, 2, 1))
print(total_X_train.shape)
print(total_y_train.shape)
print(total_X_test.shape)
print(total_y_test.shape)
print(total_X_val.shape)
print(total_y_val.shape)

Using TensorFlow backend.


Loading data pickle...
Data pickle loaded.
(11844, 100, 22)
(11844, 4)
(3101, 100, 22)
(3101, 4)
(2961, 100, 22)
(2961, 4)


In [2]:
config = {
    # Network
    'num_inputs': total_X_train.shape[0],
    'input_shape': (total_X_train.shape[1],total_X_train.shape[2],1),
    'epochs': 50,
    'dropout': 0.5,
    'batch_size': 640,
    'l2': 0.03,
    'LSTM': False,
    'lr': 0.001
}

In [3]:
ConvMixGRU = ConvMixGRU()
ConvMixGRU.build_model(config)
ConvMixGRU.train(total_X_train, total_y_train, total_X_val, total_y_val, config, workpath)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 91, 22)            4862      
_________________________________________________________________
batch_normalization_1 (Batch (None, 91, 22)            364       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 45, 22)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 45, 44)            8844      
_________________________________________________________________
dropout_1 (Dropout)          (None, 45, 44)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1980)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

11844/11844 [==============================] - 1s 98us/step - loss: 0.9671 - accuracy: 0.8267 - val_loss: 1.1327 - val_accuracy: 0.6518
Epoch 46/50
11844/11844 [==============================] - 1s 99us/step - loss: 0.9695 - accuracy: 0.8219 - val_loss: 1.1545 - val_accuracy: 0.6675
Epoch 47/50
11844/11844 [==============================] - 1s 97us/step - loss: 0.9734 - accuracy: 0.8190 - val_loss: 1.1278 - val_accuracy: 0.6733
Epoch 48/50
11844/11844 [==============================] - 1s 98us/step - loss: 0.9654 - accuracy: 0.8288 - val_loss: 1.1434 - val_accuracy: 0.6592
Epoch 49/50
11844/11844 [==============================] - 1s 97us/step - loss: 0.9541 - accuracy: 0.8399 - val_loss: 1.1380 - val_accuracy: 0.6501
Epoch 50/50
11844/11844 [==============================] - 1s 97us/step - loss: 0.9566 - accuracy: 0.8345 - val_loss: 1.1289 - val_accuracy: 0.6686


In [4]:
# Majority Voting
preds = [ConvMixGRU.predict(total_X_test[i::443], 0) for i in range(443)]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:443])]
raw = ConvMixGRU.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'ConvMixGRU.pickle')
replace_model_if_better(filepath, np.mean(result), ConvMixGRU, config)

3101/3101 [==============================] - 0s 86us/step
Raw Acc result: 0.6875656161308289
Majority Vote result: 0.7061422121896162
Old model exists. Comparing performance.
New model is worse than the old one. Will not update the old model


False

In [5]:
# Load the checkpointed model with highest val acc and perform majority voting
model_path = join(workpath, '*.hdf5')
list_of_files = glob.glob(model_path)
latest_file = max(list_of_files, key=getctime)
model_val = load_model(latest_file)
preds = [model_val.predict(total_X_test[i::total_X_test.shape[0]], verbose=0) 
         for i in range(total_X_test.shape[0])]
majority_pred = []
for pred in preds:
    arg_maxes = [np.argmax(p) for p in pred]
    votes = np.bincount(arg_maxes)
    out = np.ndarray(shape=(4,), buffer=np.zeros(4), dtype=int)
    out[votes.argmax()] = 1
    majority_pred.append(out)
result = [(a == b).all() for a, b in zip(majority_pred, total_y_test[0:total_X_test.shape[0]])]
raw = model_val.evaluate(total_X_test, total_y_test)
print("Raw Acc result: {}".format(raw[1]))
print("Majority Vote result: {}".format(np.mean(result)))
filepath = join(save_path, 'best_val', 'ConvMixGRU_val.pickle')
replaced = replace_model_if_better(filepath, np.mean(result), model_val, config)
rmtree(workpath)

3101/3101 [==============================] - 0s 104us/step
Raw Acc result: 0.6615734609847568
Majority Vote result: 0.6827284746855852
Old model exists. Comparing performance.
New model is worse than the old one. Will not update the old model
